# **TSCeption: Deep Learning Architecture for EEG Data Classification**

### Introduction
The TSCeption architecture is tailored for EEG (Electroencephalography) data classification, focusing on sequential data characteristics found in brain activity recordings. It combines convolutional and recurrent neural networks to effectively capture spatial and temporal dependencies within EEG signals.

Beginning with Conv1D layers, TSCeption extracts hierarchical features across different temporal scales, supported by batch normalization for stability. MaxPooling layers then downsample the features, enhancing computational efficiency while preserving information richness.

Incorporating an LSTM layer, TSCeption models long-term dependencies in EEG time series, critical for understanding cognitive state transitions during tasks like mental arithmetic. Dense layers with dropout regularization ensure robust classification, using softmax activation for accurate state identification.

This approach involves rigorous preprocessing, including artifact removal and feature extraction via Power Spectral Density (PSD) analysis. Supervised learning optimizes model performance metrics like accuracy, validated through thorough evaluation on independent test data. TSCeption thus offers a robust framework for EEG classification, uncovering insights into cognitive processes from brain activity data.

In [ ]:
!pip install mne
!pip install tensorflow

!pip install scikeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.7/347.7 kB 30.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.4.1 which is incompatible.


In [ ]:
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import classification_report
from tensorflow.keras import layers, models, regularizers, optimizers, callbacks

# File Paths
edf_files = {
    "background": ['/content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/Subject{:02d}_1.edf'.format(i) for i in range(1, 36)],
    "task": ['/content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/Subject{:02d}_2.edf'.format(i) for i in range(1, 36)]
}
csv_file = '/content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/subject-info.csv'

# Frequency Bands
bands = {'Delta': (1, 4), 'Theta': (4, 8), 'Alpha': (8, 12), 'Beta': (12, 30), 'Gamma': (30, 100)}

# Data Initialization
features = []
labels = []
subject_data = []

# Load Subject Info
subject_info = pd.read_csv(csv_file, delimiter='\t')

# Preprocessing & Feature Extraction
for i in range(35):
    for condition, files in edf_files.items():
        raw = mne.io.read_raw_edf(files[i], preload=True)

        # Artifact Removal (ICA)
        ica = mne.preprocessing.ICA(n_components=15, random_state=97)
        ica.fit(raw)
        raw = ica.apply(raw)

        # PSD Calculation
        psds, freqs = mne.time_frequency.psd_array_welch(raw.get_data(), sfreq=raw.info['sfreq'], fmin=1, fmax=100, n_fft=2048)

        # Band-wise Power Features
        band_powers = []
        for band, (fmin, fmax) in bands.items():
            idx_band = np.where((freqs >= fmin) & (freqs <= fmax))[0]
            band_powers.append(np.mean(psds[:, idx_band], axis=1))

        features.append(np.array(band_powers).T)
        labels.append(1 if condition == 'task' else 0)
        subject_data.append(subject_info.iloc[i].tolist())

# Convert to Arrays & Standardize
features = np.array(features)
labels = np.array(labels)
subject_data = np.array(subject_data)

# One-Hot Encode Subject Data
encoder = OneHotEncoder(handle_unknown='ignore')
subject_data_encoded = encoder.fit_transform(subject_data).toarray()

# Standardize Features
scaler = StandardScaler()
features = scaler.fit_transform(features.reshape(-1, features.shape[-1])).reshape(features.shape)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42, stratify=labels)

# Model Definition (Enhanced TSCeption with RNN & Regularization)
def build_enhanced_tsception_model(nb_classes, Chans, Samples):
    input_layer = layers.Input(shape=(Chans, Samples))

    conv1 = layers.Conv1D(filters=32, kernel_size=10, activation='relu', padding='same')(input_layer)
    conv1 = layers.BatchNormalization()(conv1)
    conv1 = layers.MaxPooling1D(pool_size=2)(conv1)

    conv2 = layers.Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(conv1)
    conv2 = layers.BatchNormalization()(conv2)
    conv2 = layers.MaxPooling1D(pool_size=2)(conv2)

    conv3 = layers.Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(conv2)
    conv3 = layers.BatchNormalization()(conv3)
    conv3 = layers.MaxPooling1D(pool_size=2)(conv3)

    conv4 = layers.Conv1D(filters=256, kernel_size=3, activation='relu', padding='same')(conv3)
    conv4 = layers.BatchNormalization()(conv4)
    conv4 = layers.MaxPooling1D(pool_size=2)(conv4)
    conv5 = layers.Conv1D(filters=512, kernel_size=3, activation='relu', padding='same')(conv4)
    conv5 = layers.BatchNormalization()(conv5)
    conv5 = layers.MaxPooling1D(pool_size=2)(conv5)

    flatten = layers.Flatten()(conv4)
    reshape = layers.Reshape((1, -1))(conv4)  # Reshape to (batch_size, 1, num_features)

    lstm = layers.LSTM(128)(reshape)
    dense1 = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001))(lstm)
    dense1 = layers.Dropout(0.5)(dense1)
    output_layer = layers.Dense(nb_classes, activation='softmax')(dense1)

    model = models.Model(inputs=input_layer, outputs=output_layer)
    return model

# Model Building & Compilation
Chans, Samples = X_train.shape[1], X_train.shape[2]
model = build_enhanced_tsception_model(2, Chans, Samples)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

# Training
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[callbacks.EarlyStopping(patience=15, restore_best_weights=True)]
)

# Evaluation and Reporting
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

y_pred = np.argmax(model.predict(X_test), axis=-1)
print(classification_report(y_test, y_pred))


Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/Subject01_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Fitting ICA to data using 21 channels (please be patient, this may take a while)


<ipython-input-31-c5939c7a778b>:35: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  ica.fit(raw)


Selecting by number: 15 components
Fitting ICA took 3.7s.
Applying ICA to Raw instance
    Transforming to ICA space (15 components)
    Zeroing out 0 ICA components
    Projecting back using 21 PCA components
Effective window size : 4.096 (s)
Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/Subject01_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Fitting ICA to data using 21 channels (please be patient, this may take a while)
Selecting by number: 15 components


<ipython-input-31-c5939c7a778b>:35: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  ica.fit(raw)


Fitting ICA took 2.4s.
Applying ICA to Raw instance
    Transforming to ICA space (15 components)
    Zeroing out 0 ICA components
    Projecting back using 21 PCA components
Effective window size : 4.096 (s)
Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/Subject02_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Fitting ICA to data using 21 channels (please be patient, this may take a while)


...

<ipython-input-31-c5939c7a778b>:35: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  ica.fit(raw)


Fitting ICA took 5.3s.
Applying ICA to Raw instance
    Transforming to ICA space (15 components)
    Zeroing out 0 ICA components
    Projecting back using 21 PCA components
Effective window size : 4.096 (s)
Extracting EDF parameters from /content/drive/MyDrive/eeg-during-mental-arithmetic-tasks-1.0.0/Subject35_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Fitting ICA to data using 21 channels (please be patient, this may take a while)
Selecting by number: 15 components


<ipython-input-31-c5939c7a778b>:35: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  ica.fit(raw)


Fitting ICA took 2.3s.
Applying ICA to Raw instance
    Transforming to ICA space (15 components)
    Zeroing out 0 ICA components
    Projecting back using 21 PCA components
Effective window size : 4.096 (s)
Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 632ms/step - accuracy: 0.4435 - loss: 0.8997 - val_accuracy: 0.5000 - val_loss: 0.8965
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.4970 - loss: 0.8944 - val_accuracy: 0.5000 - val_loss: 0.8962
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.4330 - loss: 0.8996 - val_accuracy: 0.5000 - val_loss: 0.8959
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.6801 - loss: 0.8539 - val_accuracy: 0.5000 - val_loss: 0.8957
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.6577 - loss: 0.8516 - val_accuracy: 0.7143 - val_loss: 0.8954
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.7604 - loss: 0.8319 - val_accuracy: 0.6429 - val_loss: 0.8950
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 405ms/step
              precision    recall  f1-score   support

           0       0.54      1.00      0.70         7
           1       1.00      0.14      0.25         7

    accuracy                           0.57        14
   macro avg       0.77      0.57      0.47        14
weighted avg       0.77      0.57      0.47        14

